# Using State Legislation for Latenet Dirichlet Allocation - Capstone Project

# Step 1: Acquire Data

### Using the Legiscan API to access data

LegiScan has a very useful API that I can use to get information about bills in each state.
The format I can use for this is: 

https://api.legiscan.com/?key=APIKEY&op=OPERATION&PARAMS



In [3]:
import requests
import pandas as pd
import json
import base64
import numpy as np

In [4]:
# I want to build a big function that will do the querying-converting to dataframe, labeling and saving for me.

In [433]:
##I need to query by session IDS to get the most useful resluts, but they don't follow any specific pattern.
## I will use the API to get a list of the session IDs by state.

state_list = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
             'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD',
             'TN','TX','UT','VT','VA','WA','WV','WI','WY']

len(state_list)

state_ids=[]

In [673]:
##This function generates a dataframe of state ids.  I need this to pass into the second API query function so I can
## sort my results by year

def generate_session_ids(list):
    ids_list= []
    df_state= pd.DataFrame()
    for x in list:
        api_request= 'https://api.legiscan.com/?key=e54bf8ff09109e13b9c66f08518711f4&op=getSessionList&state={}'.format(x)
        r = list_session_ids = requests.get(api_request)
        js_doc= r.json()
        i = js_doc['sessions']
        df_temp=pd.DataFrame(i)
        df_temp['state_name'] = x
        df_state= pd.concat([df_state, df_temp],axis=0)
    return df_state
        
        
        

In [163]:
## Creating the final session id dataframe
ids = generate_session_ids(state_list)
ids.reset_index(drop=True, inplace=True)

In [167]:
## Save the CSV so I don't have to keep running this query
ids.to_csv('~/Documents/GA-DSI/Capstone/state_id.csv', mode='w')

In [675]:
## Reading the dataframe back in
ids = pd.read_csv('~/Documents/GA-DSI/Capstone/state_id.csv')

In [680]:
ids_target=ids[(ids.year_start> 2006) & (ids.year_start< 2017)]

In [677]:
ids_target.head()

,Unnamed: 0,name,session_id,session_name,special,state_id,year_end,year_start,state_name
0,0,Regular Session 2017,1254,Regular Session,0,1,2017,2017,AL
1,1,First Special Session 2016,1253,1st Special Session,1,1,2016,2016,AL
2,2,Regular Session 2016,1172,Regular Session,0,1,2016,2016,AL
3,3,Second Special Session 2015,1178,2nd Special Session,1,1,2015,2015,AL
4,4,First Special Session 2015,1173,1st Special Session,1,1,2015,2015,AL


In [681]:
def get_bills():
    df_allbills= pd.DataFrame(columns=['change_hash', 'description','last_action', 'last_action_date' ,
                                       'number', 'status', 'status_date', 'title', 'url', 'session_id','state',
                                      'year_end','year_start'])
    for index, row in ids_target.iterrows():
        request_url= 'https://api.legiscan.com/?key=e54bf8ff09109e13b9c66f08518711f4&op=getMasterList&id={}'.format(row.session_id)
        r = requests.get(request_url)
        js_doc= r.json()
        item = js_doc['masterlist']
        df_temp = pd.DataFrame(item).T
        df_temp['session_id'] = row.session_id
        df_temp['state'] = row.state_name
        df_temp['year_end'] = row.year_end
        df_temp['year_start']= row.year_start
#         df_temp['session_name']=row.session_name
        df_allbills = pd.concat([df_allbills, df_temp],axis=0)
    return df_allbills


In [682]:
df_bills = get_bills()

In [683]:
df_bills.shape

(812698, 15)

In [684]:
df_test.columns

NameError: name 'df_test' is not defined

In [685]:
df_bills.isnull().sum()


bill_id                391
change_hash            391
description            892
last_action           1277
last_action_date      1460
number                 391
session_id               0
session_name        812307
state                    0
status                 391
status_date           5888
title                  391
url                    391
year_end                 0
year_start               0
dtype: int64

In [686]:
df_bills.to_csv('~/Documents/GA-DSI/Capstone/bills_final2.csv', encoding='utf-8')

In [687]:
#reimport the saved files

df=pd.read_csv('~/Documents/GA-DSI/Capstone/bills_final2.csv')

In [688]:
## It looks like there was some error in the search that returned about 242 empty bills. I'll drop those:
df = df.dropna(subset=['bill_id'])
## and my session_name column didn't work.  I will drop that too.
df = df.drop('session_name', 1)


In [689]:
df.status.value_counts()

1.0    433800
4.0    235036
6.0     74852
2.0     52887
0.0      5497
3.0      5241
5.0      4994
Name: status, dtype: int64

In [690]:
df.head(3)

,Unnamed: 0,bill_id,change_hash,description,last_action,last_action_date,number,session_id,state,status,status_date,title,url,year_end,year_start
0,0,884498.0,6dfb74a3dd3adeba02bf4b88fdfd42a8,"Fire Departments, non-volunteer, diesel exhaus...",Read for the first time and referred to the Ho...,2016-08-15,HB1,1253.0,AL,1.0,2016-08-15,"Fire Departments, non-volunteer, diesel exhaus...",https://legiscan.com/AL/bill/HB1/2016/X1,2016.0,2016.0
1,1,884496.0,1966c29ff2c50e2ce3a5a76d93d6506a,Supplemental Nutritional Assistance Program be...,Delivered to Governor at 1:17 p.m. on August 2...,2016-08-25,HB2,1253.0,AL,4.0,2016-08-25,Supplemental Nutritional Assistance Program be...,https://legiscan.com/AL/bill/HB2/2016/X1,2016.0,2016.0
2,10,884511.0,d77bb734c4a07619bf06a3a64b12cb35,Alabama Lottery and Alabama Lottery Corporatio...,Read for the first time and referred to the Ho...,2016-08-15,HB11,1253.0,AL,1.0,2016-08-15,Alabama Lottery and Alabama Lottery Corporatio...,https://legiscan.com/AL/bill/HB11/2016/X1,2016.0,2016.0


In [691]:
df['bill_id'] = df['bill_id'].astype(int)

Now I want to create a dataframe with all of the bill data, including the full text.

The API call for this is:
https://api.legiscan.com/?key=APIKEY&op=getBill&id=BILL_ID

In [319]:
def get_text():
    text_df = pd.DataFrame(columns=['bill_id','date','doc','doc_id','mime','mime_id','text_size','type','type_id'])
    temp_df = []

    for index, row in df.iterrows():
        request_url= 'https://api.legiscan.com/?key=81cd57e580151c2ea2eee5b616a4944a&op=getBillText&id={}'.format(row.bill_id)
        r = requests.get(request_url)
        js_doc= r.json()
        try:
            item = js_doc['text']
#         temp_df = []
            temp_df.append(item)
#         temp_df['bill_id'] = item['bill_id']
#         temp_df['date']=item['date']
#         temp_df['doc']=item['doc']
#         temp_df['doc_id']=item['doc_id']
#         temp_df['mime']=item['mime']
#         temp_df['mime_id']=item['mime_id']
#         temp_df['text_size']=item['text_size']
#         temp_df['type']=item['type']
#         temp_df['type_id']=item['type_id']
#         temp_df.to_csv('~/Documents/GA-DSI/Capstone/bill_list.csv', mode = 'a', encoding='utf-8')
#         text_df = pd.concat([text_df, temp_df],axis=0)
        except KeyError:
            pass 
    return text_df


In [320]:
text_final = get_text()

ConnectionError: HTTPSConnectionPool(host='api.legiscan.com', port=443): Max retries exceeded with url: /?key=81cd57e580151c2ea2eee5b616a4944a&op=getBillText&id=533112 (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x12f428050>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [704]:
df.shape

(812307, 15)

In [40]:
df.year_start.value_counts()

2015.0    160635
2011.0    158588
2013.0    157597
2009.0    113421
2010.0     53299
2014.0     53044
2016.0     52807
2012.0     50301
2007.0      6816
2008.0      5799
Name: year_start, dtype: int64

# Step 2: Transforming Bills and Clustering by Text

I need to use NLP to examine each bill and then find bills that are highly similar to group them. I can generate features from the text file, then use a some kind of clustering technique to group the bills.

Most of this code comes from the wonderful resource : http://brandonrose.org/clustering

In [13]:
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [14]:
#first I want to 
doc_complete = df.description.tolist()

In [15]:
len(doc_complete)

812307

In [16]:
doc_complete = [str(x) for x in doc_complete]

In [17]:
doc_complete = [line.decode('utf-8').strip() for line in doc_complete]

In [24]:
df.description[39994]

'An act to add Article 3.5 (commencing with Section 221.2) to Chapter 2 of Part 1 of Division 1 of Title 1 of the Education Code, relating to schools.'

In [26]:
doc_clean[39994]

[u'act',
 u'add',
 u'article',
 u'35',
 u'commencing',
 u'section',
 u'2212',
 u'chapter',
 u'part',
 u'division',
 u'title',
 u'education',
 u'code',
 u'relating',
 u'school']

In [18]:
# Now I need to clean the strings:

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

In [19]:
stop = set(stopwords.words('english'))
additional_stop_words = ['commonwealth','virginia','a','b','6','state', '1','2','3','4','5','7','8','9']
additional_stop_words = [unicode(i,'utf-8') for i in additional_stop_words]
stop = set(stopwords.words('english') + list(string.punctuation) + additional_stop_words)


exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]        


In [27]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [28]:
len(doc_term_matrix)

812307

In [29]:
doc_term_matrix[39994]

[(143, 1),
 (258, 1),
 (304, 1),
 (607, 1),
 (650, 1),
 (697, 1),
 (704, 1),
 (908, 1),
 (1000, 1),
 (1014, 1),
 (1207, 1),
 (1222, 1),
 (1231, 1),
 (1277, 1),
 (18854, 1)]

In [731]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamulticore.LdaMulticore

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50, alpha = .3, workers=7)

In [48]:
ldamodel.save('first.lda')


In [720]:
ldamodel.update(doc_term_matrix)


KeyboardInterrupt: 

In [733]:
yo = [ldamodel[x] for x in doc_term_matrix]

In [734]:
len(yo)

812307

In [735]:
len(doc_term_matrix)

812307

In [736]:
df['category']= yo

In [737]:
df.category[1]

[(0, 0.021500526164903621),
 (1, 0.022262171677617897),
 (2, 0.67021597613264461),
 (3, 0.023184646816020699),
 (4, 0.021678952779742907),
 (5, 0.021431708022539197),
 (6, 0.093972644446699907),
 (7, 0.022043385626200073),
 (8, 0.07811211652643428),
 (9, 0.025597871807196922)]

In [738]:
max(df.category[0])

(9, 0.2687905407060705)

In [739]:
df.category[0]

[(0, 0.015040820103628266),
 (1, 0.016818164229731352),
 (2, 0.04555936100603937),
 (3, 0.015496096914961099),
 (4, 0.015257933392712766),
 (5, 0.015000010287627662),
 (6, 0.015990426529470903),
 (7, 0.10209805320149135),
 (8, 0.48994859362826676),
 (9, 0.2687905407060705)]

In [740]:
from operator import itemgetter



In [741]:
def find_max_category():
    new_list =[]
    for x in df.category:
        ok_list =[]
        z=0
        y = 0
        for i in x:
            if i[1]> y:
                z = i
                y=i[1]
            ok_list.append(i[1])
        new_list.append(z)
    return new_list
            

In [742]:
x=find_max_category()

In [743]:
len(x)

812307

In [744]:
df['best_match'] = x

In [745]:
df.head()

,Unnamed: 0,bill_id,change_hash,description,last_action,last_action_date,number,session_id,state,status,status_date,title,url,year_end,year_start,category,best_match
0,0,884498,6dfb74a3dd3adeba02bf4b88fdfd42a8,"Fire Departments, non-volunteer, diesel exhaus...",Read for the first time and referred to the Ho...,2016-08-15,HB1,1253.0,AL,1.0,2016-08-15,"Fire Departments, non-volunteer, diesel exhaus...",https://legiscan.com/AL/bill/HB1/2016/X1,2016.0,2016.0,"[(0, 0.0150408201036), (1, 0.0168181642297), (...","(8, 0.489948593628)"
1,1,884496,1966c29ff2c50e2ce3a5a76d93d6506a,Supplemental Nutritional Assistance Program be...,Delivered to Governor at 1:17 p.m. on August 2...,2016-08-25,HB2,1253.0,AL,4.0,2016-08-25,Supplemental Nutritional Assistance Program be...,https://legiscan.com/AL/bill/HB2/2016/X1,2016.0,2016.0,"[(0, 0.0215005261649), (1, 0.0222621716776), (...","(2, 0.670215976133)"
2,10,884511,d77bb734c4a07619bf06a3a64b12cb35,Alabama Lottery and Alabama Lottery Corporatio...,Read for the first time and referred to the Ho...,2016-08-15,HB11,1253.0,AL,1.0,2016-08-15,Alabama Lottery and Alabama Lottery Corporatio...,https://legiscan.com/AL/bill/HB11/2016/X1,2016.0,2016.0,"[(0, 0.0579073627452), (1, 0.413713857819), (2...","(1, 0.413713857819)"
3,100,885579,d3060fe40e09dc5160140ae92deb6caa,"McCormick, Lonnie, death mourned",Joint Rule 11,2016-09-06,HR76,1253.0,AL,4.0,2016-09-06,"McCormick, Lonnie, death mourned",https://legiscan.com/AL/bill/HR76/2016/X1,2016.0,2016.0,"[(0, 0.461765483774), (1, 0.0428571518614), (2...","(0, 0.461765483774)"
4,101,885547,00db50ddde4493e069d9ca455328a7f5,"McBride, Robert Eugene, death mourned",Joint Rule 11,2016-09-06,HR77,1253.0,AL,4.0,2016-09-06,"McBride, Robert Eugene, death mourned",https://legiscan.com/AL/bill/HR77/2016/X1,2016.0,2016.0,"[(0, 0.523256487533), (1, 0.0375000061018), (2...","(0, 0.523256487533)"


In [746]:
df['first_cat'] = [x[0] for x in df.best_match]

In [747]:
df.to_csv('~/desktop/bills_labeled.csv')

Now I need to tranform my dataframe into a format I can use for poisson regression. I will sum up the type of each bill by state by year.


In [50]:
states_list = df.state.unique()

In [51]:
year_list = df.year_start.unique()

In [52]:
year_list

array([ 2016.,  2015.,  2014.,  2013.,  2012.,  2011.,  2010.,  2009.,
        2007.,  2008.])

In [751]:
cat_unique = df['first_cat'].unique

In [53]:
df.head(1)

,Unnamed: 0,Unnamed: 0.1,bill_id,change_hash,description,last_action,last_action_date,number,session_id,state,status,status_date,title,url,year_end,year_start,category,best_match,first_cat
0,0,0,884498,6dfb74a3dd3adeba02bf4b88fdfd42a8,"Fire Departments, non-volunteer, diesel exhaus...",Read for the first time and referred to the Ho...,2016-08-15,HB1,1253.0,AL,1.0,2016-08-15,"Fire Departments, non-volunteer, diesel exhaus...",https://legiscan.com/AL/bill/HB1/2016/X1,2016.0,2016.0,"[(0, 0.015040820103628266), (1, 0.016818164229...","(8, 0.48994859362826676)",8


In [669]:
df.to_csv('~/desktop/bills_labeled.csv')

In [47]:
df= pd.read_csv('~/desktop/bills_labeled.csv')

In [54]:
def create_final_frame():
    dict_list=[]
    for i in states_list:
        for x in year_list:
            for n in range(10):
                temp_dict = {}
                c = df['state'][(df['first_cat']==n) & (df['year_start']==x)].count()
                temp_dict['count'] = c 
                temp_dict['state'] = i
                temp_dict['year'] = x
                temp_dict['topic']= n
                dict_list.append(temp_dict)
    return dict_list

In [55]:
df_transformed = create_final_frame()

In [56]:
df_transformed = pd.DataFrame(df_transformed)

In [57]:
df_transformed.shape

(5000, 4)

In [59]:
#now I need to add my y value to my dataframe, which will be the counts by subject for the previous year or 2 years

def add_y():
    temp_value_list = []
    for index, x in df_transformed.iterrows():
        year_plus = x['year'] + 1
        state_temp = x.state
        topic_temp = x.topic
        new_value = df_transformed['count'].loc[(df_transformed['state']==state_temp)&(df_transformed['topic']==topic_temp)
                                           & (df_transformed['year']==year_plus)]
        try:
            new_value = new_value.item()
        except:
            new_value = None
        temp_value_list.append(new_value)
    return temp_value_list

In [60]:
df_transformed['next_year'] = add_y()

In [61]:
#let's look at two-year patterns and see if we can spot some similarities:

def add_y2():
    temp_value_list = []
    for index, x in df_transformed.iterrows():
        year_plus = x['year'] + 2
        state_temp = x.state
        topic_temp = x.topic
        new_value = df_transformed['count'].loc[(df_transformed['state']==state_temp)&(df_transformed['topic']==topic_temp)
                                           & (df_transformed['year']==year_plus)]
        try:
            new_value = new_value.item()
        except:
            new_value = None
        temp_value_list.append(new_value)
    return temp_value_list


In [62]:
df_transformed['2-year']  = add_y2()

In [63]:
df_transformed.head(50)

,count,state,topic,year,next_year,2-year
0,5656,AL,0,2016.0,NaN,NaN
1,4825,AL,1,2016.0,NaN,NaN
2,8140,AL,2,2016.0,NaN,NaN
3,5094,AL,3,2016.0,NaN,NaN
4,3966,AL,4,2016.0,NaN,NaN
5,183,AL,5,2016.0,NaN,NaN
6,6478,AL,6,2016.0,NaN,NaN
7,6313,AL,7,2016.0,NaN,NaN
8,2640,AL,8,2016.0,NaN,NaN
9,9512,AL,9,2016.0,NaN,NaN


In [64]:
# Finally, I'll add dummy variables for states

df_state = pd.get_dummies(df_transformed['state'])
df_transformed = pd.concat([df_transformed, df_state], axis=1)


In [8]:
df_transformed  = pd.read_csv('~/desktop/bills_labeled.csv')

# Step 3: Modeling

Now that I have my dataframe in the format I need, I can regress my y values to predict the outlook for each state.

For this I will have to use 

In [65]:
df_transformed_dropped = df_transformed.dropna()

In [66]:
df_transformed_dropped

,count,state,topic,year,next_year,2-year,AK,AL,AR,AZ,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY
20,5776,AL,0,2014.0,27462.0,5656.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
21,4944,AL,1,2014.0,15358.0,4825.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
22,7920,AL,2,2014.0,19548.0,8140.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
23,5722,AL,3,2014.0,13076.0,5094.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
24,3940,AL,4,2014.0,16356.0,3966.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
25,182,AL,5,2014.0,7772.0,183.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
26,6376,AL,6,2014.0,16209.0,6478.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
27,6001,AL,7,2014.0,14968.0,6313.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
28,2673,AL,8,2014.0,9507.0,2640.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
29,9510,AL,9,2014.0,20379.0,9512.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
df_transformed_dropped.next_year.mean()

9408.55

In [68]:
df_transformed_dropped.next_year.std()**2

45433687.49437357

In [69]:
df_transformed_dropped[df_transformed_dropped['count'] == 0]

,count,state,topic,year,next_year,2-year,AK,AL,AR,AZ,...,SD,TN,TX,UT,VA,VT,WA,WI,WV,WY


In [70]:
df_transformed_dropped.isnull().sum()

count        0
state        0
topic        0
year         0
next_year    0
2-year       0
AK           0
AL           0
AR           0
AZ           0
CA           0
CO           0
CT           0
DE           0
FL           0
GA           0
HI           0
IA           0
ID           0
IL           0
IN           0
KS           0
KY           0
LA           0
MA           0
MD           0
ME           0
MI           0
MN           0
MO           0
MS           0
MT           0
NC           0
ND           0
NE           0
NH           0
NJ           0
NM           0
NV           0
NY           0
OH           0
OK           0
OR           0
PA           0
RI           0
SC           0
SD           0
TN           0
TX           0
UT           0
VA           0
VT           0
WA           0
WI           0
WV           0
WY           0
dtype: int64

In [71]:
import statsmodels.api as sm

In [72]:
df_transformed_dropped.dtypes

count          int64
state         object
topic          int64
year         float64
next_year    float64
2-year       float64
AK             uint8
AL             uint8
AR             uint8
AZ             uint8
CA             uint8
CO             uint8
CT             uint8
DE             uint8
FL             uint8
GA             uint8
HI             uint8
IA             uint8
ID             uint8
IL             uint8
IN             uint8
KS             uint8
KY             uint8
LA             uint8
MA             uint8
MD             uint8
ME             uint8
MI             uint8
MN             uint8
MO             uint8
MS             uint8
MT             uint8
NC             uint8
ND             uint8
NE             uint8
NH             uint8
NJ             uint8
NM             uint8
NV             uint8
NY             uint8
OH             uint8
OK             uint8
OR             uint8
PA             uint8
RI             uint8
SC             uint8
SD             uint8
TN           

In [74]:
df_transformed_dropped['next_year']=df_transformed_dropped['next_year'].astype(int)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [75]:
df_transformed_dropped['count']=df_transformed_dropped['count'].astype(int)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [76]:
xcols = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA',
             'MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD',
             'TN','TX','UT','VT','VA','WA','WV','WI','WY', 'count']

In [85]:
xcols = ['count']

In [86]:
X = df_transformed_dropped[xcols]

In [87]:
y = df_transformed_dropped['2-year']

In [781]:
results = sm.Poisson(y, X).fit(method='nm', maxiter=5000)

Optimization terminated successfully.
         Current function value: 39820.016976
         Iterations: 17
         Function evaluations: 34


In [782]:
print results.summary()

                          Poisson Regression Results                          
Dep. Variable:                 2-year   No. Observations:                 4000
Model:                        Poisson   Df Residuals:                     3999
Method:                           MLE   Df Model:                            0
Date:                Thu, 25 May 2017   Pseudo R-squ.:                     nan
Time:                        07:35:39   Log-Likelihood:            -1.5928e+08
converged:                       True   LL-Null:                           nan
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
count          0.0005   9.53e-09   4.74e+04      0.000       0.000       0.000


In [783]:
results.resid

20        5642.368237
21        4815.643224
22        8104.050928
23        5080.697139
24        3960.058237
25         181.914201
26        6460.118368
27        6297.907969
28        2636.650002
29        9438.208936
30     -168462.199032
31       14545.830501
32       16028.692501
33       12598.638873
34       14138.278163
35        7739.879832
36       14872.074337
37       14321.088616
38        9431.691689
39       13092.727958
40        5768.644232
41        4935.008320
42        7895.447188
43        5710.364014
44        3934.271975
45         180.923492
46        6357.988497
47        5987.219460
48        2668.994723
49        9444.454879
            ...      
4970     18069.186727
4971     14718.620463
4972     16483.978712
4973     13837.223118
4974     16458.320698
4975      8459.158383
4976     15800.893317
4977     14769.093633
4978     10512.662077
4979     20906.727828
4980     19657.204546
4981     10866.834823
4982     10940.779811
4983     10363.971104
4984     1

In [89]:
from sklearn.linear_model import  LinearRegression

In [90]:
lm = LinearRegression()

In [91]:
lm_model = lm.fit(X,y)

In [92]:
from sklearn.model_selection import cross_val_score


In [93]:
scores = cross_val_score(lm, X, y, cv=8)
scores

array([ 0.6561266 ,  0.65501072,  0.64702779,  0.62152622,  0.6561266 ,
        0.65501072,  0.64702779,  0.62152622])

In [94]:
np.mean(scores)

0.64492283321245836

In [ ]:
## So I can successfully predict the number of each topic 